This concept notebook shows an example of how one might measure distances on an image.

In [ ]:
import math

from astropy import units as u
from astroquery.mast import Observations
from regions import RectanglePixelRegion, PixCoord

from jdaviz import Imviz

We will use an image with some nice features to measure.

In [ ]:
filename = 'jw01349-o058_t001_miri_f1500w_i2d.fits'
local_path = f'/path/to/{filename}'  # Change path as needed
Observations.download_file(f'mast:JWST/product/{filename}', local_path=local_path, cache=True)

In [ ]:
imviz = Imviz()
imviz.load_data(local_path)
imviz.show()

Adjust some viewer properties so features are visible and centered.

In [ ]:
imviz.default_viewer.stretch = 'arcsinh'
imviz.default_viewer.cuts = 'minmax'

In [ ]:
imviz.default_viewer.center_on((767, 643))

In [ ]:
imviz.default_viewer.zoom_level = 5

We programmatically mark features, but users usually would do this interactively, perhaps with the single-pixel subsets.

In [ ]:
center_pt = RectanglePixelRegion(PixCoord(x=770, y=638), width=1, height=1)
edge_pt = RectanglePixelRegion(PixCoord(x=786, y=661), width=1, height=1)

In [ ]:
imviz.load_regions([center_pt, edge_pt])

Now, we have to get the info back out. This would work no matter how the features are marked as long as they are subsets, and not markers.

In [ ]:
regions = imviz.get_interactive_regions()
regions

This part uses hidden methods and is not meant for end-users, but might be useful for a dev to use in internal implementations.

In [ ]:
# In a plugin, this could be the selected viewer.
viewer = imviz.default_viewer

In [ ]:
# In a plugin, this could be different depending on UI/UX.
# Here, we hardcode for now.
# In multi-image scenario, we would also need to call
# viewer._get_real_xy(selected_image, subset.center.x, subset.center.y)
# to get the correct pixel location.
x0, y0 = regions['Subset 1'].center.x, regions['Subset 1'].center.y
x1, y1 = regions['Subset 2'].center.x, regions['Subset 2'].center.y

In [ ]:
# In a plugin, this could be the selected data.
data = imviz.app.data_collection[0]

This gives you the separation in pixels. When linked by WCS and the non-reference data is selected, you will have to check that the actual pixel location is still in bounds or not. That is not done here.

In [ ]:
dx = x1 - x0
dy = y1 - y0
sep_pix = math.sqrt(dx * dx + dy * dy)
sep_pix

This gives you the separation in sky. WCS must be present for this to work. GWCS will return NaNs if pixels are outside of its bounding box, I think.

In [ ]:
sky_coords = data.coords.pixel_to_world((x0, x1), (y0, y1))

In [ ]:
sep_arcsec = sky_coords[0].separation(sky_coords[1]).to(u.arcsec)
sep_arcsec